In [ ]:
import numpy as np
str_num="0.1"
model1="TRANS"
model2="StageNet"
model=f"{str_num}margin_mimic3_"
retain_prob=np.load(f"deep_result/{model}{model1}_y_prob.npy")
trans_prob=np.load(f"deep_result/{model}{model2}_y_prob.npy")
import numpy as np


retain_true = np.load(f"deep_result/{model}{model1}_y_true.npy")
trans_true = np.load(f"deep_result/{model}{model2}_y_true.npy")
print(np.array_equal(retain_true, trans_true))


In [ ]:
import pickle
from pyhealth.tokenizer import Tokenizer
task_dataset = pickle.load(open('data/patient_time_mimic3_1/mimic3_box_dataset_1.pkl', 'rb'))

ccs9 = pickle.load(open('file/ccs9.pkl','rb'))
label_tokenizer = Tokenizer(tokens=ccs9)
ccs_idx=sorted(ccs9)



In [ ]:
import torch
def batch_to_multihot(label, num_labels: int) -> torch.tensor:

    multihot = torch.zeros((len(label), num_labels))
    for i, l in enumerate(label):
        multihot[i, l] = 1
    return multihot

labels=[['99'],['12']]

labels_index = label_tokenizer.batch_encode_2d(
    labels, padding=False, truncation=False
)
num_labels = label_tokenizer.get_vocabulary_size()
labels = batch_to_multihot(labels_index, num_labels)


In [ ]:
import torch
from torch.utils.data import Subset
from torch.utils.data import Dataset
class MMDataset(Dataset):
    def __init__(self, dataset):
        self.sequence_dataset = []
        for visit in dataset.samples:
            self.sequence_dataset.append(visit)


    def __len__(self):
        return len(self.sequence_dataset)

    def __getitem__(self, idx):
        sequence_data = self.sequence_dataset[idx]
        return sequence_data, idx
def custom_collate_fn(batch):
    sequence_data_list = [item[0] for item in batch]
    graph_data_list = [item[1] for item in batch]

    sequence_data_batch = {key: [d[key] for d in sequence_data_list if d[key]!=[]] for key in sequence_data_list[0]}

    graph_data_batch = graph_data_list

    return sequence_data_batch, graph_data_batch    

mdataset = MMDataset(task_dataset)
indices = torch.load('data/patient_time_mimic3_1/'+'/testset.pt')
testset = Subset(mdataset, indices)

In [ ]:
from datetime import datetime


time_1 = '2169-11-01 15:19'
time_2 = '2171-05-27 04:57'

time_1 = datetime.strptime(time_1, '%Y-%m-%d %H:%M')
time_2 = datetime.strptime(time_2, '%Y-%m-%d %H:%M')


time_difference = time_2 - time_1


years = time_difference.days // 365
months = (time_difference.days % 365) // 30
days = (time_difference.days % 365) % 30
hours = time_difference.seconds // 3600
minutes = (time_difference.seconds % 3600) // 60

years, months, days


In [ ]:
from torch.utils.data import DataLoader
import numpy as np
diag_test_data=[]

dataloader = DataLoader(testset, batch_size=1, shuffle=False)
y_true = retain_true
retain_prob = retain_prob
trans_prob = trans_prob
num=0
for i, (data, row1, row2, row3) in enumerate(zip(dataloader, y_true, retain_prob, trans_prob)):
    threshold=0.5
    retain_pred = (row2 >= threshold).astype(int)
    trans_pred = (row3 >= threshold).astype(int)
    t = np.where(row1 == 1)[0].tolist()  # ground truth labels index
    same_mask = (retain_pred == trans_pred)
    p = np.where(same_mask == 0)[0].tolist()


    t = [ccs_idx[j] for j in t]
    p = [ccs_idx[j] for j in p]
    
    temp=data[0].copy()
    temp['ccs_answer']=t
    temp['conflict_ccs']=p
    temp['consistent_ccs']=p
    num+=1
    diag_test_data.append(temp)
print(num)


In [ ]:
print(diag_test_data[0])

In [ ]:
import pandas as pd


df = pd.read_csv("file/ICD9CM_to_CCSCM.csv", encoding="utf-8")
df["ICD9CM"] = df["ICD9CM"].astype(str).str.replace(".", "", regex=False)

ICD9CM_to_CCSCM = dict(zip(df["ICD9CM"], df["CCSCM"]))


In [ ]:
import pandas as pd

df = pd.read_csv("file/ICD9CM.csv", encoding="utf-8")


df = df[~df["code"].astype(str).str.contains("-")]


df["code"] = df["code"].astype(str).str.replace(".", "", regex=False)
df["parent_code"] = df["parent_code"].astype(str).str.replace(".", "", regex=False)


icd_to_info = dict(zip(
    df["code"],
    df[["name", "parent_code"]].to_dict(orient="records")
))


print(icd_to_info["2143"])



In [ ]:
import pandas as pd


df = pd.read_csv("file/CCS9.csv", encoding="utf-8")


ccs_to_name = dict(zip(df["code"], df["name"]))

print(ccs_to_name[101])  

In [ ]:
from collections import defaultdict

def get_diag_ccs_ontology(hist_diag):

    ccs_icd_dict = defaultdict(list)
    for index,icd_code in enumerate(hist_diag):
        ccs_code=ICD9CM_to_CCSCM[icd_code]
        ccs_icd_dict[ccs_code].append(icd_code)
        
    final_diag_str="{"
    for ccs_code,icd_list in ccs_icd_dict.items():
        temp_str="[("
        for icd in icd_list:
            name=icd_to_info[icd]['name']
            temp_str+=f'''"{name}", '''
        temp_str=temp_str[:-2]
        temp_str+=") BELONGS TO "
        ccs_name=ccs_to_name[int(ccs_code)]
        temp_str+= f'''"{ccs_name}"'''
        temp_str+="], "
        final_diag_str+=temp_str
    final_diag_str=final_diag_str[:-2]
    final_diag_str+="}"    
    return final_diag_str
    

In [ ]:
import inflect

def number_to_capitalized_ordinal(n):
    p = inflect.engine()
    return p.ordinal(p.number_to_words(n)).capitalize()

print(number_to_capitalized_ordinal(1))  
print(number_to_capitalized_ordinal(42)) 


In [ ]:

from datetime import datetime


result = []
filt_num=0
for item in diag_test_data:

    conflict_ccs = item['conflict_ccs']
    conflict_ccs_str = ""
    candidate_idx_ccs = {}

    for index, code in enumerate(conflict_ccs):
        name = ccs_to_name[int(code)]
        conflict_ccs_str += f'''"{name}", '''
        candidate_idx_ccs[index + 1] = code
    conflict_ccs_str = conflict_ccs_str[:-2]
    conflict_ccs_idx = {v: k for k, v in candidate_idx_ccs.items()}


    hist_diag_str=""
    cur_time=item['adm_time'][-1][0]
    for num,cond_hist in enumerate(item['cond_hist']):
        hist_diag = []
        for t in cond_hist:
            hist_diag.append(t[0])
        hist_diag = list(set(hist_diag))
        if len(hist_diag)!=0:
            hist_time=item['adm_time'][num][0]
            # Define the two datetime strings
            time_1 = hist_time
            time_2 = cur_time
            
            # Convert strings to datetime objects
            time_1 = datetime.strptime(time_1, '%Y-%m-%d %H:%M')
            time_2 = datetime.strptime(time_2, '%Y-%m-%d %H:%M')
            
            # Calculate the difference between the two times
            time_difference = time_2 - time_1
            
            # Extract years, months, days, hours, and minutes from the difference
            years = time_difference.days // 365
            months = (time_difference.days % 365) // 30
            days = (time_difference.days % 365) % 30
            parts = []
            if years:
                parts.append(f"{years} years")
            if months:
                parts.append(f"{months} months")
            if days:
                parts.append(f"{days} days")
            
            if parts:
                time_str = " ".join(parts) + " ago"
            else:
                time_str = "today"
            hist_diag_str+=number_to_capitalized_ordinal(num+1)
            hist_diag_str+=f" Visit({time_str}): "

            hist_diag_str+="\n- diagnosis: "+get_diag_ccs_ontology(hist_diag)
            # hist_diag_str+="\n- ICD diagnosis: "+get_diag_str(hist_diag)
            # hist_diag_str+="\n- CCS diagnosis: "+get_ccs_str(hist_diag)
            hist_diag_str+="\n\n"

    instruction=("You are a medical diagnosis expert. Your task is to identify which diseases (CCS categories) the patient is most likely to have this time, based on:\n"
                 +"- The patient’s past diagnosis history\n"
                +"- The provided candidate CCS categories"
                  f"\n\nYour task:\n- From the candidate CCS categories, select all that are likely to be the patient’s current diseases."
                 +"\n- There may be multiple correct answers."
                  +"\n- If none of the candidate diseases are likely, answer 'None'."
                   +"\nDirectly output the answer without any explanation."
                  "\nOutput format:\nAnswer: <CCS name 1>, <CCS name 2>, ...\n\n---\n")

    input_="Patient history summary:\n"+hist_diag_str+"\n\n"
    
    input_+="Candidate diseases:\n{"+f"{conflict_ccs_str}"+"}\n\n"
    
    ccs_icd_dict = defaultdict(list)
    



        
    answer_ccs=item['ccs_answer']


    temp={'visit_id':item['visit_id'],'patient_id':item['patient_id'],"instruction":instruction,
          "input":input_,"ccs_answer":item['ccs_answer'],"conflict_ccs":conflict_ccs}
    result.append(temp)




In [ ]:
import json
temp=result[1]
print(temp['instruction']+temp['input'])
# print(str(json.dumps(temp, ensure_ascii=False, indent=2)))

In [ ]:
import json
with open(f"dataset/{str_num}margin_llm_patient_time_m3_1_v1_{model1}_{model2}.json", "w", encoding="utf-8") as f:

    json.dump(result, f, ensure_ascii=False, indent=2)